<div style="
    display: block;
    padding: 12px 20px;
    background-color: #1A73E8;
    color: white;
    border-radius: 30px;
    font-family: 'Helvetica Neue', Arial, sans-serif;
    font-size: 16px;
    font-weight: 600;
    margin: 15px auto;
    width: fit-content;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
    text-align: center;
    letter-spacing: 0.5px;
">
    <strong>MISTRAL - TEXT GENERATION</strong>
</div>

<div style="
    display: block;
    padding: 12px 20px;
    background-color: #66BB6A;
    color: white;
    border-radius: 30px;
    font-family: 'Helvetica Neue', Arial, sans-serif;
    font-size: 16px;
    font-weight: 600;
    margin: 15px auto;
    width: fit-content;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
    text-align: center;
    letter-spacing: 0.5px;
">
    <strong>Simon-Pierre Boucher</strong>
</div>

<div style="
    display: block;
    padding: 12px 20px;
    background-color: #FFA726;
    color: white;
    border-radius: 30px;
    font-family: 'Helvetica Neue', Arial, sans-serif;
    font-size: 16px;
    font-weight: 600;
    margin: 15px auto;
    width: fit-content;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
    text-align: center;
    letter-spacing: 0.5px;
">
    <strong>2024-09-14</strong>
</div>


Here's a summary of what the Python code is doing:

1. **Environment Setup**:
   - The code loads environment variables from a `.env` file using `load_dotenv()`. It then retrieves the Mistral API key from these environment variables (`MISTRAL_API_KEY`).

2. **Mistral API Text Generation Function**:
   - The function `generate_mistral_text()` is designed to send a request to the Mistral AI API to generate text.
   - Parameters such as `model`, `messages`, `temperature`, `top_p`, `max_tokens`, etc., allow customization of the request (e.g., model choice, randomness of output, and token limits).
   - The API key is passed via a header, and the request body (`data`) contains the text generation parameters, such as the input messages and various tuning options.
   - If optional parameters (`max_tokens`, `stop`, etc.) are provided, they are included in the request.
   - The function handles any API errors with exception handling and returns the response as a JSON dictionary.

3. **Response Formatting**:
   - The `format_mistral_response()` function extracts the relevant part of the API response, specifically the assistant's reply, and formats it using Markdown for clearer output (e.g., the assistant's message is bolded).

4. **Example Use**:
   - The code sets up a specific `model` (`mistral-small-latest`) and a user message asking about quantum entanglement.
   - It sends the message to the Mistral API using the `generate_mistral_text()` function with parameters such as `temperature` and `max_tokens`.
   - The response is formatted using the `format_mistral_response()` function, and the output is printed.

This code demonstrates how to interact with the Mistral API for text generation in a conversational context.

In [2]:
import os
import requests
from dotenv import load_dotenv
from IPython.display import display, HTML
import re

# Charger les variables d'environnement depuis le fichier .env
load_dotenv()
# Obtenir la clé API depuis les variables d'environnement
api_key = os.getenv("MISTRAL_API_KEY")

In [3]:
import requests

def generate_mistral_text(api_key, model, messages, temperature=0.7, top_p=1.0, max_tokens=None, min_tokens=None, stream=False, stop=None, random_seed=None, response_format=None, tools=None, tool_choice="auto", safe_prompt=False):
    """
    Generate text using Mistral's API.
    
    Parameters:
    - api_key (str): The API key for Mistral.
    - model (str): The model to use for text generation.
    - messages (list): A list of messages to pass to the API in a conversation format.
    - temperature (float): Controls randomness in the output (0-1.5).
    - top_p (float): Nucleus sampling (0-1).
    - max_tokens (int): The maximum number of tokens to generate in the completion.
    - min_tokens (int): The minimum number of tokens to generate in the completion.
    - stream (bool): Whether to stream back partial progress.
    - stop (str or list): Stop generation if this token or one of these tokens is detected.
    - random_seed (int): The seed to use for random sampling.
    - response_format (dict): The format of the response.
    - tools (list): A list of tools to use.
    - tool_choice (str): Tool choice for the response ("auto", "none", "any").
    - safe_prompt (bool): Whether to inject a safety prompt before all conversations.
    
    Returns:
    - response (dict): The API response as a dictionary.
    """
    url = "https://api.mistral.ai/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    data = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
        "top_p": top_p,
        "stream": stream,
        "tool_choice": tool_choice,
        "safe_prompt": safe_prompt
    }

    # Optional parameters
    if max_tokens is not None:
        data["max_tokens"] = max_tokens
    if min_tokens is not None:
        data["min_tokens"] = min_tokens
    if stop is not None:
        data["stop"] = stop
    if random_seed is not None:
        data["random_seed"] = random_seed
    if response_format is not None:
        data["response_format"] = response_format
    if tools is not None:
        data["tools"] = tools

    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

In [4]:
def format_mistral_response(response):
    """
    Formats the response from Mistral API to display only the assistant's message.
    
    Parameters:
    - response (dict): The API response as a dictionary.
    
    Returns:
    - formatted_text (str): A formatted string with Markdown for the assistant's message.
    """
    if response and "choices" in response:
        # Extract the assistant's message from the response
        assistant_message = response.get("choices", [])[0].get("message", {}).get("content", "No reply found.")
        formatted_text = f"**Assistant:**\n\n{assistant_message}\n"
        return formatted_text
    else:
        return "No valid response received."


In [5]:
model = "mistral-small-latest"
messages = [
    {"role": "user", "content": "Explain the concept of quantum entanglement and how it challenges classical notions of locality and realism. What are the implications of entanglement for our understanding of causality and information transfer?"}
]
response = generate_mistral_text(api_key, model, messages, temperature=0.7, max_tokens=2000, top_p=0.9)
formatted_response = format_mistral_response(response)
print(formatted_response)

**Assistant:**

Quantum entanglement is a fascinating phenomenon in quantum physics where two or more particles become linked and instantaneously affect each other's state no matter the distance between them. This means that the state of one entangled particle cannot be described independently of the state of the other, even if they are light-years apart.

This concept challenges classical notions of locality and realism. Locality, in this context, refers to the idea that an object is directly influenced only by its immediate surroundings. However, with entanglement, a particle's state can be affected by a distant particle without any physical interaction between them, seemingly violating locality. Realism, on the other hand, is the idea that particles have definite properties or states that exist independently of measurement. But in the quantum world, a particle doesn't have a definite state until it's measured, and the act of measurement can instantly change the state of its entangle

In [6]:
model = "mistral-medium-latest"
messages = [
    {"role": "user", "content": "Explain the concept of quantum entanglement and how it challenges classical notions of locality and realism. What are the implications of entanglement for our understanding of causality and information transfer?"}
]
response = generate_mistral_text(api_key, model, messages, temperature=0.7, max_tokens=2000, top_p=0.9)
formatted_response = format_mistral_response(response)
print(formatted_response)

**Assistant:**

Quantum entanglement is a phenomenon in quantum mechanics where two or more particles become linked and instantaneously affect each other's state, regardless of the distance between them. This means that the properties of one entangled particle, such as its position, momentum, or spin, are directly connected to those of another entangled particle.

This concept challenges classical notions of locality and realism in several ways:

1. Locality: In classical physics, objects can only be influenced by their immediate surroundings. However, quantum entanglement defies this principle, as entangled particles seem to instantaneously influence each other's properties over vast distances. This apparent violation of locality has been experimentally confirmed through various tests, including the famous Bell test experiments.

2. Realism: The idea of realism suggests that objects have definite properties that exist independently of our observation or measurement. Quantum entangleme

In [7]:
model = "mistral-large-latest"
messages = [
    {"role": "user", "content": "Explain the concept of quantum entanglement and how it challenges classical notions of locality and realism. What are the implications of entanglement for our understanding of causality and information transfer?"}
]
response = generate_mistral_text(api_key, model, messages, temperature=0.7, max_tokens=2000, top_p=0.9)
formatted_response = format_mistral_response(response)
print(formatted_response)

**Assistant:**

Quantum entanglement is a fascinating phenomenon in quantum physics where two or more particles become linked and instantaneously affect each other's state, no matter the distance between them. Here's a breakdown of the concept and its implications:

1. **Quantum Entanglement**: When two particles, such as photons or electrons, become entangled, their properties (like spin or polarization) become correlated in such a way that the state of one particle cannot be described independently of the other. This means that measuring a property of one particle instantly determines the corresponding property of the other, regardless of the distance between them.

2. **Challenges to Classical Notions**:

   - **Locality**: In classical physics, an object is influenced only by its immediate surroundings. Entanglement challenges this notion because the particles seem to affect each other instantaneously across vast distances, a phenomenon Einstein famously referred to as "spooky acti

In [8]:
model = "open-mistral-7b"
messages = [
    {"role": "user", "content": "Explain the concept of quantum entanglement and how it challenges classical notions of locality and realism. What are the implications of entanglement for our understanding of causality and information transfer?"}
]
response = generate_mistral_text(api_key, model, messages, temperature=0.7, max_tokens=2000, top_p=0.9)
formatted_response = format_mistral_response(response)
print(formatted_response)

**Assistant:**

Quantum entanglement is a fascinating phenomenon in quantum mechanics, a branch of physics that describes nature at its smallest scales. Two or more particles can become entangled, meaning their properties become interconnected in such a way that the state of one particle instantly affects the state of the other, no matter the distance between them. This effect is often referred to as "spooky action at a distance" by Albert Einstein, who found it difficult to reconcile with his classical notions of reality.

Classically, we expect that physical properties of objects are local and independent. For example, if you have two coins, the state of one coin (heads or tails) does not affect the state of the other coin unless they are physically interacting. However, in quantum entanglement, this is not the case. If two particles are entangled, measuring the property of one particle instantly determines the property of the other, regardless of the distance between them. This viol

In [9]:
model = "open-mixtral-8x7b"
messages = [
    {"role": "user", "content": "Explain the concept of quantum entanglement and how it challenges classical notions of locality and realism. What are the implications of entanglement for our understanding of causality and information transfer?"}
]
response = generate_mistral_text(api_key, model, messages, temperature=0.7, max_tokens=2000, top_p=0.9)
formatted_response = format_mistral_response(response)
print(formatted_response)

**Assistant:**

Quantum entanglement is a quantum mechanical phenomenon where two or more particles become linked and instantaneously affect each other's state no matter how far apart they are. This means that the state of one entangled particle cannot be described independently of the state of the other, even when they are separated by large distances.

This concept challenges classical notions of locality and realism. Locality, in the context of physics, refers to the idea that an object is only directly influenced by its immediate surroundings. However, quantum entanglement seems to violate this principle as changes in one entangled particle's state appear to affect the other instantaneously, regardless of the distance between them.

Realism, on the other hand, is the belief that physical properties exist independently of observation. But in quantum mechanics, the act of measurement affects the outcome, suggesting that physical properties may not be well-defined until they are measu

In [10]:
model = "open-mixtral-8x22b"
messages = [
    {"role": "user", "content": "Explain the concept of quantum entanglement and how it challenges classical notions of locality and realism. What are the implications of entanglement for our understanding of causality and information transfer?"}
]
response = generate_mistral_text(api_key, model, messages, temperature=0.7, max_tokens=2000, top_p=0.9)
formatted_response = format_mistral_response(response)
print(formatted_response)

**Assistant:**

Quantanglement is a fascinating phenomenon in quantum mechanics where two or more particles become interconnected and instantaneously affect each other's state, no matter how far apart they are. This correlation between entangled particles persists even when the particles are separated by large distances, which seems to defy classical notions of locality and realism.

Classical notions of locality imply that objects can only be influenced by their immediate surroundings, and changes in one object cannot propagate faster than the speed of light. However, quantum entanglement challenges this idea, as the state of one entangled particle is immediately connected to the state of its partner, regardless of the distance separating them. This "spooky action at a distance," as Einstein famously called it, appears to violate the principle of locality.

Moreover, quantum entanglement also challenges classical notions of realism, which assume that objects have definite properties i